In [1]:
from qiskit_nature.drivers import PySCFDriver,UnitsType

molecule = 'Li 0.0 0.0 0.0; H 0.0 0.0 1.5474'
distance_unit = UnitsType.ANGSTROM
basis = "sto3g"

driver = PySCFDriver(molecule,unit = distance_unit,basis = basis)
molecule = driver.run()
h1 = molecule.one_body_integrals
print(h1)

[[-4.73853724  0.10753914  0.1675853   0.          0.         -0.03026284
   0.          0.          0.          0.          0.          0.        ]
 [ 0.10753914 -1.51317577  0.03434669  0.          0.         -0.06802917
   0.          0.          0.          0.          0.          0.        ]
 [ 0.1675853   0.03434669 -1.12916229  0.          0.          0.03143223
   0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.         -1.14077094  0.          0.
   0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.         -1.14077094  0.
   0.          0.          0.          0.          0.          0.        ]
 [-0.03026284 -0.06802917  0.03143223  0.          0.         -0.9418187
   0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
  -4.73853724  0.10753914  0.1675853   0.          0.    

In [2]:
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit_nature.transformers import FreezeCoreTransformer

#Freeze the core and remove two unoccupied orbitals(find unoccupied ones from h1) to reduce number of qubits
# Freezing core refers to excluding inner-shells 
problem = ElectronicStructureProblem(driver, q_molecule_transformers=[FreezeCoreTransformer(freeze_core=True, remove_orbitals=[3,4])])
# Generate the second-quantized operators
second_q_ops = problem.second_q_ops()

# Hamiltonian
main_op = second_q_ops[0] 

In [3]:
from qiskit_nature.mappers.second_quantization import ParityMapper
from qiskit_nature.converters.second_quantization.qubit_converter import QubitConverter

#Parity fermion to qubit mapping
#Two_qubit_reduction is set to true to carry out two qubit reduction
converter = QubitConverter(mapper=ParityMapper(),two_qubit_reduction=True)

# The fermionic operators are mapped to qubit operators
num_particles = (problem.molecule_data_transformed.num_alpha,
             problem.molecule_data_transformed.num_beta)
qubit_op = converter.convert(main_op, num_particles=num_particles)


In [4]:
from qiskit_nature.circuit.library import HartreeFock

num_particles = (problem.molecule_data_transformed.num_alpha,
             problem.molecule_data_transformed.num_beta)
num_spin_orbitals =2* problem.molecule_data_transformed.num_molecular_orbitals
init_state = HartreeFock(num_spin_orbitals, num_particles, converter)
print(init_state)

     ┌───┐
q_0: ┤ X ├
     ├───┤
q_1: ┤ X ├
     └───┘
q_2: ─────
          
q_3: ─────
          


In [5]:
from qiskit.circuit.library import TwoLocal

# Two Local reduces cost of circuit compared to UCCSD and SUCCD 
rotation_blocks = ['ry'] # Gates used in the rotation layer
entanglement_blocks = ['cx'] # Gates used in the entanglement layer
entanglement = [[3, 0], [0, 1],[1,2]] # manual entanglement; circular works too but 4 cnots are implemented   

repetitions = 1 # Repetitions of rotation_blocks and entanglement_blocks
skip_final_rotation_layer = True # No rotation layer is added at the end.
ansatz =TwoLocal(qubit_op.num_qubits, rotation_blocks, entanglement_blocks, reps=repetitions, 
                     entanglement=entanglement, skip_final_rotation_layer = skip_final_rotation_layer)
ansatz.compose(init_state, front=True, inplace=True)
    
print(ansatz) 

        ┌───┐    ┌──────────┐┌───┐          
q_0: ───┤ X ├────┤ RY(θ[0]) ├┤ X ├──■───────
        ├───┤    ├──────────┤└─┬─┘┌─┴─┐     
q_1: ───┤ X ├────┤ RY(θ[1]) ├──┼──┤ X ├──■──
     ┌──┴───┴───┐└──────────┘  │  └───┘┌─┴─┐
q_2: ┤ RY(θ[2]) ├──────────────┼───────┤ X ├
     ├──────────┤              │       └───┘
q_3: ┤ RY(θ[3]) ├──────────────■────────────
     └──────────┘                           


In [6]:
from qiskit import Aer
backend = Aer.get_backend('statevector_simulator')

In [7]:
from qiskit.algorithms.optimizers import SLSQP
# SLSQP and COBYLA are used for statevector simulator 
# SPSA used for noisy devices 
optimizer = SLSQP(maxiter = 500)

In [8]:
from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory
from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
import numpy as np 

def exact_diagonalizer(problem, converter):
    solver = NumPyMinimumEigensolverFactory()
    calc = GroundStateEigensolver(converter, solver)
    result = calc.solve(problem)
    return result

result_exact = exact_diagonalizer(problem, converter)
exact_energy = np.real(result_exact.eigenenergies[0])
print("Exact electronic energy", exact_energy)
print(result_exact)

Exact electronic energy -1.0887060157347395
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -8.907396311316
  - computed part:      -1.088706015735
  - FreezeCoreTransformer extracted energy part: -7.818690295581
~ Nuclear repulsion energy (Hartree): 1.025934879643
> Total ground state energy (Hartree): -7.881461431673
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  2.92416221]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  4.76300889]
    - computed part:      [0.0  0.0  4.76695575]
    - FreezeCoreTransformer extracted energy part: [0.0  0.0  -0.00394686]
  > Dipole moment (a.u.): [0.0  0.0  -1.83884668]  Total: 1.83884668
                 (debye): [0.0  0.0  -4.67388163]  Total: 4.67388163
 


In [9]:
from qiskit.algorithms import VQE
from IPython.display import display, clear_output

# Print and save the data in lists
def callback(eval_count, parameters, mean, std):  
    # Overwrites the same line when printing
    display("Evaluation: {}, Energy: {}, Std: {}".format(eval_count, mean, std))
    clear_output(wait=True)
    counts.append(eval_count)
    values.append(mean)
    params.append(parameters)
    deviation.append(std)

counts = []
values = []
params = []
deviation = []

# Set initial parameters of the ansatz
# We choose a fixed small displacement 
# So all participants start from similar starting point
try:
    initial_point = [0.01] * len(ansatz.ordered_parameters)
except:
    initial_point = [0.01] * ansatz.num_parameters

algorithm = VQE(ansatz,
                optimizer=optimizer,
                quantum_instance=backend,
                callback=callback,
                initial_point=initial_point)

result = algorithm.compute_minimum_eigenvalue(qubit_op)

print(result)

{   'aux_operator_eigenvalues': None,
    'cost_function_evals': 90,
    'eigenstate': array([ 1.04127571e-03-1.27519497e-19j, -7.56856894e-06+1.68055990e-21j,
       -1.98129959e-05+8.21864479e-38j,  9.92042742e-01-3.41767936e-16j,
        5.20015455e-02-6.36835262e-18j, -3.77976050e-04+8.39275428e-20j,
       -3.96734198e-07+1.64569632e-39j,  1.98646022e-02-6.84353990e-18j,
       -2.25757512e-03+7.77755588e-19j,  4.50881041e-08-1.87030328e-40j,
        4.29562755e-05-9.53820923e-21j, -5.90987898e-03+7.23751438e-19j,
       -1.12743814e-01+3.88412906e-17j,  2.25171015e-06-9.34033704e-39j,
        8.60153791e-07-1.90992509e-22j, -1.18339049e-04+1.44923538e-20j]),
    'eigenvalue': -1.0861926894966019,
    'optimal_parameters': {   ParameterVectorElement(θ[2]): 9.384735436329942,
                              ParameterVectorElement(θ[3]): -0.2263252179688979,
                              ParameterVectorElement(θ[1]): 3.1408306379708377,
                              ParameterVectorEle

In [11]:
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Unroller

# Unroller transpile your circuit into CNOTs and U gates
pass_ = Unroller(['u', 'cx'])
pm = PassManager(pass_)
ansatz_tp = pm.run(ansatz)
cnots = ansatz_tp.count_ops()['cx']
score = cnots

accuracy_threshold = 4.0 # in mHa
energy = result.optimal_value

result_dict = {
        'optimizer': optimizer.__class__.__name__,
        'mapping': converter.mapper.__class__.__name__,
        'ansatz': ansatz.__class__.__name__,
        'rotation blocks': rotation_blocks,
        'entanglement_blocks': entanglement_blocks,
        'entanglement': entanglement,
        'repetitions': repetitions,
        'skip_final_rotation_layer': skip_final_rotation_layer,
        'energy (Ha)': energy,
        'error (mHa)': (energy-exact_energy)*1000,
        'pass': (energy-exact_energy)*1000 <= accuracy_threshold,
        '# of parameters': len(result.optimal_point),
        'final parameters': result.optimal_point,
        '# of evaluations': result.optimizer_evals,
        'optimizer time': result.optimizer_time,
        '# of qubits': int(qubit_op.num_qubits),
        '# of CNOTs': cnots,
        'score': score}

# Display and save the data
import pandas as pd
import os.path
filename = 'results_h.csv'
if os.path.isfile(filename):
    result_df = pd.read_csv(filename)
    result_df = result_df.append([result_dict])
else:
    result_df = pd.DataFrame.from_dict([result_dict])
result_df.to_csv(filename)
result_df[['optimizer','ansatz', '# of qubits', '# of parameters','rotation blocks', 'entanglement_blocks',
    'entanglement', 'repetitions', 'error (mHa)', 'pass', 'score']]

,optimizer,ansatz,# of qubits,# of parameters,rotation blocks,entanglement_blocks,entanglement,repetitions,error (mHa),pass,score
0,SLSQP,TwoLocal,4,4,[ry],[cx],"[[3, 0], [0, 1], [1, 2]]",1,2.513326,True,3
